In [1]:
## CSV Files downloaded from https://www.data.gouv.fr/fr/datasets/repertoire-national-des-associations/  Fichier RNA Waldec du 01 Mars 2022

import findspark
findspark.init()
from pyspark import SparkContext
sc = SparkContext(appName="LIst of associations")

22/03/24 21:36:36 WARN Utils: Your hostname, DESKTOP-3481LPC resolves to a loopback address: 127.0.1.1; using 172.21.11.8 instead (on interface eth0)
22/03/24 21:36:36 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/03/24 21:36:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [16]:
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("how to read csv file") \
    .getOrCreate()

file_location = "/mnt/d/Projects/mongulu/acu-te/tests/data/rna_waldec_20220301/"
file_type = "csv"

# CSV options
infer_schema = "false"
first_row_is_header = "true"
delimiter = ";"

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .option("encoding", "ISO-8859-1") \
  .load(file_location)

df.show(5)
df.count()

+----------+------+-----+------+-------+----------+----------+----------+----------+------+----------+--------------------+--------------------+--------------------+-------------+-------------+---------------+------------+---------------+-------------+----------------+------------+--------------+---------------+--------------------+--------------+--------------+---------------+--------------------+------------+---------------+--------------------+---------+------------+-------+--------+--------------------+--------+-------------------+
|        id| id_ex|siret|rup_mi|gestion|date_creat|date_decla|date_publi|date_disso|nature|groupement|               titre|         titre_court|               objet|objet_social1|objet_social2|adrs_complement|adrs_numvoie|adrs_repetition|adrs_typevoie|    adrs_libvoie|adrs_distrib|adrs_codeinsee|adrs_codepostal|     adrs_libcommune|adrg_declarant|adrg_complemid|adrg_complemgeo|        adrg_libvoie|adrg_distrib|adrg_codepostal|       adrg_achemine|adrg_pa

2119460

In [54]:
df.columns

['id',
 'id_ex',
 'siret',
 'rup_mi',
 'gestion',
 'date_creat',
 'date_decla',
 'date_publi',
 'date_disso',
 'nature',
 'groupement',
 'titre',
 'titre_court',
 'objet',
 'objet_social1',
 'objet_social2',
 'adrs_complement',
 'adrs_numvoie',
 'adrs_repetition',
 'adrs_typevoie',
 'adrs_libvoie',
 'adrs_distrib',
 'adrs_codeinsee',
 'adrs_codepostal',
 'adrs_libcommune',
 'adrg_declarant',
 'adrg_complemid',
 'adrg_complemgeo',
 'adrg_libvoie',
 'adrg_distrib',
 'adrg_codepostal',
 'adrg_achemine',
 'adrg_pays',
 'dir_civilite',
 'siteweb',
 'publiweb',
 'observation',
 'position',
 'maj_time']

In [168]:
from pyspark.sql.functions import col,lower,upper,lit,concat,when,concat_ws
from pyspark import StorageLevel

# use concat_ws instead of concat to avoid value being null if one is null (concat)
df2 = df.filter(df.titre.contains("CAMEROUN") | df.objet.contains("cameroun") ) \
        .withColumn("titre",upper(col("titre")))\
        .withColumn("objet",lower(col("objet")))\
        .select("id","titre", "objet", "objet_social1","objet_social2","adrs_numvoie","adrs_typevoie","adrs_libvoie","adrs_codepostal","adrs_libcommune", "siteweb") \
        .withColumn("adrs",concat_ws(" ",col("adrs_numvoie"), col("adrs_typevoie"),col("adrs_libvoie"),col('adrs_codepostal'),col('adrs_libcommune'))) \
        .cache()

df2.show()
df2.count()


+----------+--------------------+--------------------+-------------+-------------+------------+-------------+--------------------+---------------+------------------+--------------------+--------------------+
|        id|               titre|               objet|objet_social1|objet_social2|adrs_numvoie|adrs_typevoie|        adrs_libvoie|adrs_codepostal|   adrs_libcommune|             siteweb|                adrs|
+----------+--------------------+--------------------+-------------+-------------+------------+-------------+--------------------+---------------+------------------+--------------------+--------------------+
|W751075369|ASSOCIATION NDOG-...|créer et favorise...|       014000|       000000|          94|          RUE|    des Chataigniers|          95100|        Argenteuil|                null|94 RUE des Chatai...|
|W751101199|CERCLE D'ETUDES D...|étudier les tradi...|       020015|       000000|         148|           AV|            d'Italie|          75013|             Paris|   

22/03/25 21:57:42 WARN CacheManager: Asked to cache already cached data.


1213

In [170]:
df3 = df2.filter(col("objet_social1").isNull())
df3.show()
df3.count()

+----------+--------------------+--------------------+-------------+-------------+------------+-------------+------------+---------------+---------------+-------+----+
|        id|               titre|               objet|objet_social1|objet_social2|adrs_numvoie|adrs_typevoie|adrs_libvoie|adrs_codepostal|adrs_libcommune|siteweb|adrs|
+----------+--------------------+--------------------+-------------+-------------+------------+-------------+------------+---------------+---------------+-------+----+
|W751226452|MOUVEMENT POUR LA...|oeuvrer à la rech...|         null|         null|        null|         null|        null|           null|           null|   null|    |
|W751242998|CAMEROON DIASPORA...|fédérer et agir s...|         null|         null|        null|         null|        null|           null|           null|   null|    |
|W751257781|HAUT CONSEIL DES ...|fédérer les citoy...|         null|         null|        null|         null|        null|           null|           null|   nul

49

In [152]:
# https://journal-officiel-datadila.opendatasoft.com/explore/dataset/jo_associations/api/?sort=dateparution&q=W751226452
import requests
#import requests_cache

#requests_cache.install_cache('demo_cache')

import json

def get_info (rna):
    url = "https://journal-officiel-datadila.opendatasoft.com/api/records/1.0/search/?dataset=jo_associations&sort=dateparution&facet=localisation_facette&facet=domaine_activite_categorise"
    r = requests.get('{}&q={}'.format(url,rna))
    info = {"objet_social1": 000000, "objet_social2":00000, "adrs":""}

    try:
        siege_social = json.loads( r.json()["records"][0]['fields']['contenu'])['assoLoi1901']\
                           .get('creation',{"siegeSocial": {"complGeographique": ""}})\
                           .get('siegeSocial',{"complGeographique": ""})
        if "###" in  r.json()["records"][0]['fields']['domaine_activite_categorise']:
            info = {
                "objet_social1" : r.json()["records"][0]['fields']['domaine_activite_categorise'].split("###")[0].split("/")[0] ,
                "objet_social2" : r.json()["records"][0]['fields']['domaine_activite_categorise'].split("###")[1].split("/")[1] ,
                "adrs" : siege_social.get('complGeographique','') + " " + siege_social.get('codePostal', '') + " " + siege_social.get('commune','')
            }
        else:
            info = {
                "objet_social1" : r.json()["records"][0]['fields']['domaine_activite_categorise'].split("/")[0] ,
                "objet_social2" : r.json()["records"][0]['fields']['domaine_activite_categorise'].split("/")[1] ,
                "adrs" : siege_social.get('complGeographique','') + " " + siege_social.get('codePostal', '') + " " + siege_social.get('commune','')
            }
    except Exception as e :
        print(rna)

    return info


def get_info2(rna):
    url = "https://entreprise.data.gouv.fr/api/rna/v1/id/W751226452"
    r = requests.get('{}&q={}'.format(url,rna))
    info = {"objet_social1": 000000, "objet_social2":00000, "adrs":""}

print(get_info("W751226452"))

from pyspark.sql.functions import udf

from pyspark.sql.types import StringType
udf_func = udf(lambda x: get_info(x)["objet_social1"], StringType())
udf_func2 = udf(lambda x: get_info(x)["objet_social2"],  StringType())
udf_func3 = udf(lambda x: get_info(x)["adrs"], StringType())

# df4 = df3.withColumn('objet_social1', udf_func(df.id) ) \
#          .withColumn('objet_social2', udf_func2(df.id) ) \
#          .withColumn('adrs', udf_func3(df.id) )
# df4.show(49)
#df3.select("id").toPandas().apply(lambda row: get_info(row["id"])["objet_social1"] , axis=1)
# df4 W543014141

{'objet_social1': '14000', 'objet_social2': '14035', 'adrs': '66, avenue Champs Elysées, 75008 Paris 8e'}


In [ ]:
df5 = df2.withColumn("objet_social1",when(df2.objet_social1.isNull() ,lit(udf_func(df2.id))).otherwise(df2.objet_social1) ) \
         .withColumn("objet_social2",when(df2.objet_social2.isNull() ,lit(udf_func2(df2.id))).otherwise(df2.objet_social2) ) \
         .withColumn("adrs",when(df2.adrs.isNull() ,lit(udf_func3(df2.id))).otherwise(df2.adrs) )
df5.show()
df5.persist(StorageLevel.MEMORY_ONLY)

In [22]:
from pyspark.sql.functions import length
df5.where(length(col("objet_social1")) >=7).show()
df5.where(length(col("objet_social1")) >=7).count()

W751105303>                                                         (0 + 1) / 1]
W751105303
W751118437
W751118437
W751128115
W751128115
W751135798
W751135798
W751155603
W751155603
W751159475
W751159475
W751167344
W751167344
W751176298
W751176298
W751191215
W751191215
W751209557>                                                         (0 + 1) / 1]
W751209557
W751226253
W751226253
W751235451>                                                         (0 + 1) / 1]
W751235451
W751244071
W751244071
W751258323>                                                         (0 + 1) / 1]
W751258323
W591008409
W591008409
W595000027
W595000027
W595000074
W595000074
W595000205
W595000205
W595005789
W595005789
W595007532
W595007532
W595009028
W595009028
W595010182
W595010182
W595010704
W595010704
W595012656
W595012656
W595012868
W595012868
ERROR:root:KeyboardInterrupt while sending command.                 (0 + 1) / 1]
Traceback (most recent call last):
  File "/home/bafou/.local/share/virtualenvs/data-mvNq

KeyboardInterrupt: 

In [49]:
import pandas as pd
from pyspark.sql.types import StringType
from pyspark.sql.functions import length
waldec_csv = pd.read_csv('rna-nomenclature-waldec.csv', delimiter=";",
                         index_col=2).to_dict()["Libellé objet social parent"]
# pprint(waldec_csv)
waldec_csv[00000] = "AUTRES"
waldec_csv[19060] = "INTERVENTIONS SOCIALES"
waldec_csv[22010] = "REPRÉSENTATION, PROMOTION ET DÉFENSE D'INTÉRÊTS ÉCONOMIQUES"
waldec_csv[6035]  = "CULTURE, PRATIQUES D'ACTIVITÉS ARTISTIQUES, PRATIQUES CULTURELLES"
waldec_csv[40510] = "ACTIVITÉS RELIGIEUSES, SPIRITUELLES OU PHILOSOPHIQUES"
waldec_csv[17035] = "SANTÉ"

udf_func3 = udf(lambda x: waldec_csv[int(x or 0)] if len(x) < 7 else " ", StringType())

df6 = df5.withColumn("objet_social1_libelle",udf_func3(df5.objet_social1)) \
         .withColumn("objet_social2_libelle",udf_func3(df5.objet_social2))
df6.show()
df6.persist(StorageLevel.MEMORY_ONLY)

+----------+--------------------+--------------------+-------------+-------------+------------+-------------+--------------------+---------------+------------------+--------------------+--------------------+---------------------+---------------------+
|        id|               titre|               objet|objet_social1|objet_social2|adrs_numvoie|adrs_typevoie|        adrs_libvoie|adrs_codepostal|   adrs_libcommune|             siteweb|                adrs|objet_social1_libelle|objet_social2_libelle|
+----------+--------------------+--------------------+-------------+-------------+------------+-------------+--------------------+---------------+------------------+--------------------+--------------------+---------------------+---------------------+
|W751075369|ASSOCIATION NDOG-...|créer et favorise...|       014000|       000000|          94|          RUE|    des Chataigniers|          95100|        Argenteuil|                null|94 RUE des Chatai...| AMICALES, GROUPEM...|               

22/03/24 22:46:33 WARN CacheManager: Asked to cache already cached data.


DataFrame[id: string, titre: string, objet: string, objet_social1: string, objet_social2: string, adrs_numvoie: string, adrs_typevoie: string, adrs_libvoie: string, adrs_codepostal: string, adrs_libcommune: string, siteweb: string, adrs: string, objet_social1_libelle: string, objet_social2_libelle: string]

In [50]:
# Print files to CSV
#df6.toPandas().to_csv("ref-rna-real-mars-2022.csv")

In [153]:
df7 = df6.toPandas()
df7.loc[df7.adrs == " ", "adrs"] = df7[df7.adrs == " "].apply(lambda row: get_info(row["id"])["adrs"],axis=1)
df7.to_csv("ref-rna-real-mars-2022.csv")

W751258323
W595005789
W313001103
W931002605
W302000324
W241000045
